In [1]:
import jinja2
import yaml
import json
from pprint import pprint

In [2]:
def qstr(inp):
    if inp is None:
        inp = ""
    return json.dumps(str(inp))

def str_(inp):
    return qstr(inp)[1:-1]

def json_dumps(inp):
    return json.dumps(inp, ensure_ascii=False)

JAML_FILTERS = {
    'qstr': qstr,
    'str': str_,
    'json.dumps': json_dumps,
}

class Environment(jinja2.Environment):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filters.update(JAML_FILTERS)

env = Environment()

def convert(template, context):
    jinja_template = env.from_string(template)
    rendered = jinja_template.render(context)
    return yaml.safe_load()

In [3]:
T = '''
types:
    str:
        str: {{ 'a x:"7\nx' | qstr }}
        int: {{ 7 | qstr }}
        none: {{ None | qstr }}
        optional: {{ (None | qstr) if False else None }}
        default: {{ (None | qstr) if False else ('oops\n"' | qstr) }}
        empty: {{ "" | qstr }}
        inline: "a {{ 'x:7\nx' | str }}"
        as_object_key: { {{ "str\nas\nkey:oops'\\"" | qstr }}: 1 }
    
    int:
        int: {{ 7 | int }}
        str: {{ "7" | int }}
        optional: {{ None | int }}

    objects:
        as_json_dict_with_commas: {
            k1: v1,
            k2: v2,
        }
        as_yaml_dict_without_commas:
            k1: v1
            k2: v2
        object_by_json: {{ {'a': 1, 3: (1, 2, 3) } | json.dumps }}
        
    list:
        as_json_list: [
            x,
            { a: 1},
            1,
        ]
        as_yaml_list:
            - x
            - { a: 1}
            - 1

jinja:
    comment: test {# jinja #} comment
    cycle:
        {% for x in [1, 2, 3] %}
            - {
                k: {{ x }},
                v: 1,
            }
        {% endfor %}

    inline_if:
        {{ 'in_key' if True else 'oops' }}: in_key
        in_value: {{ 'oops' if False else 'in_value' }}
        {{ 'full: full' if True else 'oops: oops' }}

    if:
        {% if True %}in_key{% else %}oops{% endif %}: in_key
        in_value: {% if False %}oops{% else %}in_value{% endif %}
        {% if True %}full: full{% else %}oops: oops{% endif %}

'''

# template = env.from_string(T)
context = {'x': 7, 'a': '7', 'items': [{ 'k:1\n': 1}]}

# y = template.render(context)
# print(y)
# x = yaml.safe_load(y)
# print(type(x))
pprint(convert(T, context))

{'jinja': {'comment': 'test  comment',
           'cycle': [{'k': 1, 'v': 1}, {'k': 2, 'v': 1}, {'k': 3, 'v': 1}],
           'if': {'full': 'full', 'in_key': 'in_key', 'in_value': 'in_value'},
           'inline_if': {'full': 'full',
                         'in_key': 'in_key',
                         'in_value': 'in_value'}},
 'types': {'int': {'int': 7, 'optional': 0, 'str': 7},
           'list': {'as_json_list': ['x', {'a': 1}, 1],
                    'as_yaml_list': ['x', {'a': 1}, 1]},
           'objects': {'as_json_dict_with_commas': {'k1': 'v1', 'k2': 'v2'},
                       'as_yaml_dict_without_commas': {'k1': 'v1', 'k2': 'v2'},
                       'object_by_json': {'3': [1, 2, 3], 'a': 1}},
           'str': {'as_object_key': {'str\nas\nkey:oops\'"': 1},
                   'default': 'oops\n"',
                   'empty': '',
                   'inline': 'a x:7\nx',
                   'int': '7',
                   'none': '',
                   'optional': 'Non

In [23]:
x = yaml.dump({'x': 'a x:7\nx', 'y': 7}, default_flow_style=False, explicit_start=False, default_style='"')
print(x)
print(yaml.safe_load(x))

"x": "a x:7\nx"
"y": !!int "7"

{'x': 'a x:7\nx', 'y': 7}


In [167]:
T = '''
a: {{ x | qstr }}
b: "{{ x | str }}{{ y | str }}"
'''.strip()

template = env.from_string(T)
context = {'x': 1, 'y': "asdf\nxzcv"}

y = template.render(context)
print(y)
x = yaml.safe_load(y)
print(type(x))
pprint(x)

a: "1"
b: "1asdf\nxzcv"
<class 'dict'>
{'a': '1', 'b': '1asdf\nxzcv'}
